In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures
import time

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [ ]:
## soup 2
start = time.perf_counter()
def get_info(stock):
    temp_frame=pd.DataFrame()
    tick = yf.Ticker(stock)
    tick_hist = tick.history(start=start_date,end=end_date)
    if len(tick_hist.index) != 0:
        if tick.info['country'] == 'United States':
            if tick_hist['Volume'].mean()>= 10000:
                for need in hist_need:
                    temp_frame[need+' of '+stock]=tick_hist[need]
                return (temp_frame,stock)
    return pd.DataFrame(),""
start_date = '2021-07-02'
end_date = '2021-10-22'
hist_data = pd.DataFrame()
clean_stocks = []
hist_need = ['Open','High','Low','Close','Volume']
tick_data = pd.read_csv("Tickers.csv",names=["Tickers"])
#with concurrent.futures.ThreadPoolExecutor() as executor:
list_tick = []
for x in range(len(tick_data.index)):
    list_tick.append(tick_data['Tickers'].iloc[x])
print(list_tick)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_info, list_tick)
    for result in results:
        hist_data = pd.concat([hist_data,result[0]],join='outer',axis=1)
        clean_stocks.append(result[1])
    
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds(s)')
hist_data.dropna(inplace=True)
print(clean_stocks)
hist_data.head()

['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PCLN', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'SHOP.TO', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TD.TO', 'TGT', 'TWX', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']
- AGN: No data found, symbol may be delisted


In [3]:
while True:
    if "" not in clean_stocks:
        break
    else:
        clean_stocks.remove("")
print(clean_stocks)
monthly_returns = pd.DataFrame()
for stock in clean_stocks:
    monthly_returns['Monthly Returns of '+stock] = hist_data['Close of '+stock].pct_change()*100
monthly_returns.dropna(inplace=True)
monthly_returns.head()

['AAPL', 'ABBV', 'ABT', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,Monthly Returns of BIIB,Monthly Returns of BK,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-06,1.471841,0.486248,0.660795,-2.996878,4.692707,0.629081,-0.228153,-2.624553,-1.098299,-3.415298,...,-4.940534,0.178772,-1.535144,-0.718435,-0.458275,-0.525472,0.219863,-0.404951,-0.113459,-2.709750
2021-07-07,1.795526,0.881364,0.883694,0.386187,0.566963,0.837459,-1.846363,-0.798599,3.027044,-0.020208,...,-1.572021,0.081111,-1.449386,-0.310134,0.855578,-0.742674,0.341241,2.028508,1.405652,-0.035701
2021-07-08,-0.919973,-0.342615,-0.508889,-4.124817,0.942218,-2.035319,2.152906,-2.440249,3.861200,-2.303961,...,-0.488914,-0.226939,-0.556491,-0.809573,0.416077,-0.811463,-0.415389,-4.383626,-1.022119,-1.696733
2021-07-09,1.305505,0.197679,0.402476,4.480600,-0.323466,2.650744,1.191026,3.249103,-2.950815,3.289202,...,2.161804,-0.129972,3.677358,0.958130,0.000000,1.078416,0.526891,1.534306,0.872351,3.506540
2021-07-12,-0.420381,0.900663,-0.776679,1.216133,-0.021241,0.965453,-0.542595,1.473522,-2.546347,0.700982,...,0.096184,0.536845,0.447218,0.105440,1.472360,0.867185,0.953630,-0.212007,-0.275804,1.070743


In [4]:
beta_data = pd.DataFrame()
MarketIndex='^GSPTSE'
marketing_tick = yf.Ticker(MarketIndex)
marketing_hist = marketing_tick.history(start=start_date,end=end_date)
beta_data[MarketIndex] = marketing_hist['Close'].pct_change()*100
beta_data = pd.concat([beta_data, monthly_returns],join='inner',axis=1)
MarketVar= beta_data['^GSPTSE'].var()
Beta = pd.DataFrame(beta_data.cov() / MarketVar)
beta_vals = pd.DataFrame(Beta.iloc[[0]])
beta_vals.rename(index={'^GSPTSE': 'Beta'})
beta_vals.head()

,^GSPTSE,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,Monthly Returns of BIIB,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
^GSPTSE,1.0,1.30269,0.413482,0.364003,2.092086,1.032426,1.727356,1.858664,1.883553,0.003967,...,2.428232,0.212841,1.949599,0.733672,1.014328,1.173331,1.058612,1.254119,0.669891,1.837344


In [5]:
correlation_values = pd.DataFrame(monthly_returns.corr())
correlation_values.head()

,Monthly Returns of AAPL,Monthly Returns of ABBV,Monthly Returns of ABT,Monthly Returns of AIG,Monthly Returns of AMZN,Monthly Returns of AXP,Monthly Returns of BA,Monthly Returns of BAC,Monthly Returns of BIIB,Monthly Returns of BK,...,Monthly Returns of SLB,Monthly Returns of SO,Monthly Returns of SPG,Monthly Returns of T,Monthly Returns of TGT,Monthly Returns of TXN,Monthly Returns of UNH,Monthly Returns of UNP,Monthly Returns of UPS,Monthly Returns of USB
Monthly Returns of AAPL,1.000000,0.318023,0.323352,0.312570,0.482592,0.385410,0.273465,0.264024,0.195729,0.296811,...,0.122022,0.240007,0.437226,0.410392,0.425576,0.315896,0.459222,0.413400,0.353515,0.266568
Monthly Returns of ABBV,0.318023,1.000000,0.208383,0.206090,0.238915,0.258852,0.162149,0.265296,0.260402,0.308866,...,0.189968,0.130398,0.080086,0.472028,0.201192,0.252978,0.402336,0.264811,0.235714,0.289612
Monthly Returns of ABT,0.323352,0.208383,1.000000,-0.103547,0.197585,-0.077468,-0.150257,-0.121351,0.194529,-0.041574,...,-0.181944,0.411580,-0.071825,0.166170,0.262793,0.206312,0.368904,0.081460,0.251059,-0.142188
Monthly Returns of AIG,0.312570,0.206090,-0.103547,1.000000,0.019819,0.650355,0.496899,0.779164,-0.165142,0.750009,...,0.536342,0.153636,0.537380,0.566203,0.188080,0.377124,0.289966,0.570082,0.270433,0.750046
Monthly Returns of AMZN,0.482592,0.238915,0.197585,0.019819,1.000000,0.222964,0.318963,0.149246,0.159312,0.038817,...,0.131076,0.003642,0.202460,0.173548,0.325342,0.216874,0.124677,0.087267,0.254567,0.097493


In [6]:
monthly_returns['Monthly Returns of AAPL'].std(axis=0,skipna=True)

1.2797325890784927

In [7]:
list_std = []
for stock in clean_stocks:
    list_std.append(("STD of "+stock, monthly_returns['Monthly Returns of '+stock].std(axis=0,skipna=True)))
pd.DataFrame(list_std).head()

,0,1
0,STD of AAPL,1.279733
1,STD of ABBV,1.257231
2,STD of ABT,1.041513
3,STD of AIG,1.879931
4,STD of AMZN,1.563713


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.